# Project 17: Analysis of Online Reviews of Emergency apps using NLP and Technology Acceptance Models (TAM).

In this project, we collect online reviews from Apple and Google Play stores from two emergency apps(Emergency Plus , First Aid: American Red Cross), to understand how users react to them.

## Task 1 and Task 2 : Preprocessing

In [5]:
from google_play_scraper import reviews_all
import pandas as pd
from deep_translator import GoogleTranslator
from app_store_scraper import AppStore
import numpy as np

df_emergencyplus_google = pd.DataFrame()
df_emergencyplus_apple = pd.DataFrame()

df_firstaid_google = pd.DataFrame()
df_firstaid_apple = pd.DataFrame()

df_emergencyplus_combined = pd.DataFrame()
df_firstaid_combined = pd.DataFrame()

def PrepareDataForTranslation(reviews_dataframe):
    global valueToFillForEmptyReviews
    valueToFillForEmptyReviews = "No review Comment"
    splitCount =400;
    reviews_dataframe['Review'] = reviews_dataframe['Review'].fillna(valueToFillForEmptyReviews)
    reviews_dataframe['Review'].str.replace('|', '')
    if(len(reviews_dataframe)< 400):
        splitCount=len(reviews_dataframe)

    return np.array_split(reviews_dataframe, splitCount)

def TranslationMechanism(splittedReview):
    concatednatedReviewString = "|".join(splittedReviewList['Review'])
    if (len(concatednatedReviewString)>5000):
        return;

def TranslateReviewContent(splittedReviewList):
    translatedReviews =[]
    concatednatedReviewString=""
    for i in range(len(splittedReviewList)):
        concatednatedReviewString = "|".join(splittedReviewList[i]['Review'])
        translatedValue = translator.translate(concatednatedReviewString, dest='en')
        x= translatedValue.split("|")
        translatedReviews = translatedReviews + x
    return translatedReviews

emergencyApps = [{'appName': 'Emergency Plus','googleId': 'com.threesixtyentertainment.nesn', 'googleLanguage': 'en', 'googleCountry': 'us', 
                                    'appStoreName':'emergency-plus','appleAppId' : '691814685', 'appleCountry': 'au'},
                  {'appName': 'Red Cross First Aid','googleId': 'com.cube.arc.fa', 'googleLanguage': 'en', 'googleCountry': 'us', 
                                    'appStoreName':'first-aid-american-red-cross','appleAppId' : '529160691', 'appleCountry': 'us'}
]

headers = ["User Name" ,"Review","Rating", "Date of Review", "reviewCreatedVersion"] 
headers_a = ["User Name" ,"Review","Rating", "Date of Review"] 

translator = GoogleTranslator(source='auto', target='en')

for app in emergencyApps: 
  
    googleReviewResult = reviews_all( app['googleId'], 
                       lang= app['googleLanguage'], 
                       country=app['googleCountry'], 
                       )

    googleAppReviewsdf = pd.DataFrame(googleReviewResult)
    googleAppReviewsdf.rename(columns = {'reviewId':'Review Id','userName':'User Name', 'content':'Review', 'score':'Rating', 'at':'Date of Review'}, inplace = True)
    splittedReviewList = PrepareDataForTranslation(googleAppReviewsdf)
    googleAppReviewsdf['Review'] = TranslateReviewContent(splittedReviewList);
    googleAppReviewsdf['Review'].replace([valueToFillForEmptyReviews], '')
    googleAppReviewsdf.rename(columns = {'reviewId':'Review Id','userName':'User Name', 'content':'Review', 'score':'Rating', 'at':'Date of Review'}, inplace = True)
    googleAppReviewsdf = googleAppReviewsdf[headers]
    
    appleApp = AppStore(app_name=app['appStoreName'], app_id = app['appleAppId'], country=app['appleCountry'])
    appleApp.review(how_many=2000)
    appleAppReviewsdf = pd.DataFrame(appleApp.reviews)
    appleAppReviewsdf.dropna(how='all')
    appleAppReviewsdf.rename(columns = {'userName':'User Name', 'review':'Review', 'rating':'Rating', 'date':'Date of Review'}, inplace = True)
    splittedAppleReviewList = PrepareDataForTranslation(appleAppReviewsdf)
    appleAppReviewsdf['Review'] = TranslateReviewContent(splittedAppleReviewList);
    appleAppReviewsdf = appleAppReviewsdf[headers_a]
    
    googleAppReviewsdf['Review'].replace([valueToFillForEmptyReviews], '')

    combinedReviewsdf = pd.concat([googleAppReviewsdf, appleAppReviewsdf])
    combinedReviewsdf['Date of Review'] = combinedReviewsdf['Date of Review'].dt.date
    
    if(app['appName'] =="Emergency Plus"):
        df_emergencyplus_google = googleAppReviewsdf
        df_emergencyplus_apple = appleAppReviewsdf
        df_emergencyplus_combined = combinedReviewsdf
    elif(app['appName'] =="Red Cross First Aid"):
        df_firstaid_google = googleAppReviewsdf
        df_firstaid_apple = appleAppReviewsdf
        df_firstaid_combined = combinedReviewsdf
    
    combinedReviewsdf.to_csv(app['appName']+'.csv', index=None, columns = headers, header=True)
    print(app['appName']+".csv file is created sucessfully. Check the files folder in left side bar. If you can't see yet refresh the folder.")

C:\Users\Kalpani Ranasinghe\AppData\Local\Temp\ipykernel_17508\2253289219.py:21: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  reviews_dataframe['Review'].str.replace('|', '')
2022-10-27 20:18:41,319 [INFO] Base - Initialised: AppStore('au', 'emergency-plus', 691814685)
2022-10-27 20:18:41,320 [INFO] Base - Ready to fetch reviews from: https://apps.apple.com/au/app/emergency-plus/id691814685
2022-10-27 20:18:42,097 [INFO] Base - [id:691814685] Fetched 201 reviews (201 fetched in total)
C:\Users\Kalpani Ranasinghe\AppData\Local\Temp\ipykernel_17508\2253289219.py:21: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  reviews_dataframe['Review'].str.replace('|', '')


Emergency Plus.csv file is created sucessfully. Check the files folder in left side bar. If you can't see yet refresh the folder.


IncompleteRead: IncompleteRead(0 bytes read)

# Task 3

In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
        
def sentimentAnalysis(app_review_df):
    reviews = app_review_df['Review'].values.tolist();
    analyzer = SentimentIntensityAnalyzer()
    sentiments = []

    for sentence in reviews:
        vs = analyzer.polarity_scores(sentence)

        #print(vs["compound"])
        if(vs["compound"] >= 0.05):
            sentiments.append(1) 
        elif(vs["compound"] > -0.05):
            sentiments.append(0)
        elif(vs["compound"] <= -0.05):
            sentiments.append(-1) 

    app_review_df["sentiment"] = sentiments
    
    return app_review_df


df_emergencyplus_google_s = sentimentAnalysis(df_emergencyplus_google)
df_emergencyplus_apple_s = sentimentAnalysis(df_emergencyplus_apple)

df_firstaid_google_s = sentimentAnalysis(df_firstaid_google)
df_firstaid_apple_s = sentimentAnalysis(df_firstaid_apple)

df_emergencyplus_combined_s = pd.concat([df_emergencyplus_google, df_emergencyplus_apple])
df_firstaid_combined_s = pd.concat([df_firstaid_google, df_firstaid_apple])

In [ ]:
df_emergencyplus_google_s.head()
df_emergencyplus_apple_s.head()

df_firstaid_google_s.head()
df_firstaid_apple_s.head()

df_emergencyplus_combined_s.szie
df_firstaid_combined_s.size

# Task 4

In [ ]:
import matplotlib.pyplot as plt
from datetime import datetime


#Emergency plus
score = df_emergency_plus['score'].values.tolist()

dates = df_emergency_plus['at'].values.tolist()
datetime_objs = []

for date in dates:
    datetime_objs.append(datetime.strptime(date, '%Y-%m-%d %H:%M:%S'))

plt.figure(figsize=(8, 6))
plt.plot(datetime_objs, score)

# Task 5

# Task 6

# Task 7